# 700

In [3]:
# Importing required Libraries
import pandas as pd
import math
import numpy as np
from seqeval.metrics import f1_score
from seqeval.metrics import classification_report,accuracy_score,f1_score
import torch.nn.functional as F
import torch
import os
from tqdm import tqdm,trange
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import json
import random
import logging
import re
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam
from keras.preprocessing.text import Tokenizer 

tags_vals = ['Empty','UNKNOWN','Email Address', 'Links', 'Skills', 'Graduation Year', 'College Name', 'Degree', 'Companies worked at', 'Location', 'Name', 'Designation', 'projects', 'Years of Experience', 'Can Relocate to', 'Rewards and Achievements', 'Address', 'University', 'Relocate to', 'Certifications', 'state', 'links', 'College', 'training', 'des', 'abc']

data_file_address = "/Users/cosmos/Desktop/DeepBlue/BACKEND/ML/Datasets (UNCLEANED)/pre_trained-700.json"
# Reading data
df_data = pd.read_json(data_file_address, lines=True)
df_data=df_data.drop(['extras'],axis=1)

# Removing New Line characters
for i in range(len(df_data)):
    df_data["content"][i] = df_data["content"][i].replace("\n", " ")
df_data.head()

#config
MAX_LEN = 300
bs = 16
device = torch.device("cpu")

# device = torch.device("cuda")
# n_gpu = torch.cuda.device_count()
# torch.cuda.get_device_name(0) 

In [4]:
# JSON formatting functions
def convert_dataturks_to_spacy(dataturks_JSON_FilePath):
    try:
        training_data = []
        lines=[]
        with open(dataturks_JSON_FilePath, 'r') as f:
            lines = f.readlines()

        for line in lines:
            data = json.loads(line)
            text = data['content'].replace("\n", " ")
            entities = []
            data_annotations = data['annotation']
            if data_annotations is not None:
                for annotation in data_annotations:
                    #only a single point in text annotation.
                    point = annotation['points'][0]
                    labels = annotation['label']
                    # handle both list of labels or a single label.
                    if not isinstance(labels, list):
                        labels = [labels]

                    for label in labels:
                        point_start = point['start']
                        point_end = point['end']
                        point_text = point['text']
                        
                        lstrip_diff = len(point_text) - len(point_text.lstrip())
                        rstrip_diff = len(point_text) - len(point_text.rstrip())
                        if lstrip_diff != 0:
                            point_start = point_start + lstrip_diff
                        if rstrip_diff != 0:
                            point_end = point_end - rstrip_diff
                        entities.append((point_start, point_end + 1 , label))
            training_data.append((text, {"entities" : entities}))
        return training_data
    except Exception as e:
        logging.exception("Unable to process " + dataturks_JSON_FilePath + "\n" + "error = " + str(e))
        return None

def trim_entity_spans(data: list) -> list:
    """Removes leading and trailing white spaces from entity spans.

    Args:
        data (list): The data to be cleaned in spaCy JSON format.

    Returns:
        list: The cleaned data.
    """
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])
    return cleaned_data        

data = trim_entity_spans(convert_dataturks_to_spacy(data_file_address))

### Changing data to appropriate format so as to feed it to the model


In [5]:
from tqdm import tqdm_notebook as tqdm
cleanedDF = pd.DataFrame(columns=["setences_cleaned"])
sum1 = 0
for i in tqdm(range(len(data))):
    start = 0
    emptyList = ["Empty"] * len(data[i][0].split())
    numberOfWords = 0
    lenOfString = len(data[i][0])
    strData = data[i][0]
    strDictData = data[i][1]
    lastIndexOfSpace = strData.rfind(' ')
    for i in range(lenOfString):
        if (strData[i]==" " and strData[i+1]!=" "):
            for k,v in strDictData.items():
                for j in range(len(v)):
                    entList = v[len(v)-j-1]
                    if (start>=int(entList[0]) and i<=int(entList[1])):
                        emptyList[numberOfWords] = entList[2]
                        break
                    else:
                        continue
            start = i + 1  
            numberOfWords += 1
        if (i == lastIndexOfSpace):
            for j in range(len(v)):
                    entList = v[len(v)-j-1]
                    if (lastIndexOfSpace>=int(entList[0]) and lenOfString<=int(entList[1])):
                        emptyList[numberOfWords] = entList[2]
                        numberOfWords += 1
    cleanedDF = cleanedDF.append(pd.Series([emptyList],  index=cleanedDF.columns ), ignore_index=True )
    sum1 = sum1 + numberOfWords

/var/folders/2h/lw5rv90j0ln7v084d1vkrcg80000gn/T/ipykernel_61726/1427337158.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(len(data))):


  0%|          | 0/701 [00:00<?, ?it/s]

In [6]:
cleanedDF.head()

,setences_cleaned
0,"[Name, Name, Empty, Empty, Empty, Empty, Empty..."
1,"[Name, Name, Designation, Designation, Designa..."
2,"[Name, Name, Designation, Empty, Empty, Empty,..."
3,"[Name, Name, Designation, Designation, Designa..."
4,"[Name, Name, Designation, Empty, Empty, Empty,..."


In [7]:
# import matplotlib.pyplot as plt
# totalNumWords = [len(one_comment.split()) for one_comment in df_data["content"]]
# plt.hist(totalNumWords)
# plt.show()

In [8]:
#tokenizer = BertTokenizer.from_pretrained('bert-large-uncased', do_lower_case=True)  #BERT
tokenizer = Tokenizer(num_words=20000) #SIMPLE

tokenizer.fit_on_texts(df_data["content"])

In [9]:
tokenized_texts = tokenizer.texts_to_sequences(df_data["content"])
# tokenized_texts = [tokenizer.tokenize(sent) for sent in df_data["content"]]
print(tokenized_texts[0])

[4659, 4660, 627, 493, 6, 4661, 1422, 5, 1131, 34, 5971, 21, 55, 50, 10, 12, 12, 17, 19, 4659, 4660, 5972, 75, 4295, 3, 506, 104, 85, 24, 1, 3395, 315, 3, 427, 273, 14, 494, 1, 26, 1242, 289, 171, 536, 5973, 5, 420, 1243, 1, 26, 3, 617, 5, 104, 26, 26, 22, 627, 493, 6, 4661, 1422, 5, 1131, 34, 270, 522, 1567, 382, 214, 1112, 32, 9018, 3396, 1567, 1, 433, 9019, 9020, 4296, 2, 797, 523, 235, 2, 871, 96, 56, 3397, 5974, 4297, 3925, 101, 421, 6, 2055, 5, 54, 139, 2860, 89, 1645, 1645, 21, 81, 24, 151, 59, 48, 27, 34, 558, 59, 48, 27, 34, 341, 59, 48, 27, 34, 69, 403, 59, 48, 27, 34, 69, 161, 59, 48, 27, 34, 95, 54, 63, 24, 2, 488, 417, 99, 99, 165, 276, 1267, 2, 169, 369, 558, 920, 2, 283, 116, 161, 858, 161, 105, 375, 341, 2, 151, 134, 403, 134, 98, 105, 156, 88, 1490, 893, 39, 37, 12, 17, 19, 4659, 4660, 5972, 41, 42, 18, 43, 18, 36, 33, 5]


In [10]:
input_ids = pad_sequences(tokenized_texts,
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [11]:
tag2idx = {t: i for i, t in enumerate(tags_vals)} #list of tokens

In [12]:
labels = cleanedDF["setences_cleaned"].tolist() #labeling the tokens of tagging

In [13]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["Empty"], padding="post",
                     dtype="long", truncating="post")

#tagging then tokenization

In [14]:
attention_masks = [[float(i>0) for i in ii] for ii in input_ids]

In [15]:
#splitting of dataset
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags, 
                                                            random_state=2018, test_size=0.1)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)


In [16]:
#tranning dataset - final - 3 each
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

- tr_inputs = content
- tr_maks = starting id
- tr_inputs = ending id

In [17]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags) #tr_inputs = content
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [18]:
model = BertForTokenClassification.from_pretrained("bert-large-uncased", num_labels=len(tag2idx))
model.cuda();

  0%|          | 4177920/1248501532 [00:04<15:20, 1351320.68B/s]

KeyboardInterrupt: 

## Fine Tunning
#### MAX_LEN = 300 & bs = 16


In [ ]:
epochs = 5 #tune accordinling
max_grad_norm = 1.0 #helps to avoid gradient exploding #1 deaulft -1 means no clipping

FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5) #1e-6 & 6e-5 also good

In [ ]:
from seqeval.metrics import f1_score

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
for _ in trange(epochs, desc="Epoch"):
    # TRAIN loop
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # forward pass
        loss = model(b_input_ids, token_type_ids=None,
                     attention_mask=b_input_mask, labels=b_labels)
        # backward pass
        loss.backward()
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        model.zero_grad()
    # print train loss per epoch
    print('########################### TRANNING ###########################')
    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    # VALIDATION on validation set
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():
            tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                                  attention_mask=b_input_mask, labels=b_labels)
            logits = model(b_input_ids, token_type_ids=None,
                           attention_mask=b_input_mask)
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.append(label_ids)
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy
        
        nb_eval_examples += b_input_ids.size(0)
        nb_eval_steps += 1
    eval_loss = eval_loss/nb_eval_steps
    print('########################### TESTING ###########################')
    print("Validation loss: {}".format(eval_loss))
    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]
    valid_tags = [tags_vals[l_ii] for l in true_labels for l_i in l for l_ii in l_i]
    print("F1-Score: {}".format(f1_score(pred_tags, valid_tags)))

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Train loss: 0.8511637701438024


Epoch:  20%|██        | 1/5 [00:08<00:34,  8.66s/it]

Validation loss: 0.5098541527986526
Validation Accuracy: 0.9231944444444444
F1-Score: 0.9242331288343558
Train loss: 0.5241081989728488


Epoch:  40%|████      | 2/5 [00:16<00:25,  8.43s/it]

...

Validation loss: 0.515742152929306
Validation Accuracy: 0.9231944444444444
F1-Score: 0.9242331288343558
Train loss: 0.5097572069901687


Epoch:  80%|████████  | 4/5 [00:32<00:08,  8.15s/it]

Validation loss: 0.48067526519298553
Validation Accuracy: 0.9231944444444444
F1-Score: 0.9242331288343558
Train loss: 0.484199753174415


Epoch: 100%|██████████| 5/5 [00:40<00:00,  8.06s/it]

Validation loss: 0.5480965077877045
Validation Accuracy: 0.9231944444444444
F1-Score: 0.9242331288343558


### SAVE

In [ ]:
#M0
torch.save(
    {
        "epoch": epochs,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
    },
    'model_e10.tar',
)

#M1
torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/Models/DeepBlue/Auto Tokenizer/')

#M2
# save
output_model = '/content/drive/MyDrive/Colab Notebooks/Models/DeepBlue/Auto Tokenizer/large-uncased-bert-300.pth'

def save(model, optimizer):
    # save
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, output_model)

save(model, optimizer)

#load
checkpoint = torch.load(output_model, map_location='cpu')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])


#M3
from transformers import AutoTokenizer
from pytorch_pretrained_bert import BertForSequenceClassification

tokenizer.save_pretrained("/content/drive/MyDrive/Colab Notebooks/Models/DeepBlue/Auto Tokenizer/large-uncased-bert-30")

model.save_pretrained("/content/drive/MyDrive/Colab Notebooks/Models/DeepBlue/Auto Tokenizer/large-uncased-bert-30")

#M4
import json
json.dump(model.to_json(), open("/content/drive/MyDrive/Colab Notebooks/Models/DeepBlue/Auto Tokenizer/large-uncased-bert-300.json", "w"))


# STATE_DICT = torch.load('../input/ner-resume/model_e6.tar', map_location=DEVICE)
# TOKENIZER = BertTokenizerFast('../input/bert-base-uncased/vocab.txt', lowercase=True)
# MODEL = BertForTokenClassification.from_pretrained(MODEL_PATH, state_dict=STATE_DICT['model_state_dict'], num_labels=12)
# #M5
# torch.save(
#     {'model_state_dict': STATE_DICT['model_state_dict']},
#     'state.bin'
# )

# #M6
# joblib.dump({'model_state_dict': STATE_DICT['model_state_dict']}, 'state.gz', compress=3)


## PREDICTION

In [ ]:
import textract
import re
from tika import parser 
fname = '/content/drive/MyDrive/Colab Notebooks/Models/DeepBlue/Test- Resumes/Resumes/Resume 2 pdf/type_14.pdf'

# text1 = textract.process('/content/drive/MyDrive/Colab Notebooks/Models/DeepBlue/Test- Resumes/Profile.pdf')
# text2 = textract.process('/content/drive/MyDrive/Colab Notebooks/Models/DeepBlue/Test- Resumes/test.pdf')

raw = parser.from_file(fname)
print(raw['content'])

####################
text = raw['content']

In [ ]:
from transformers import BertForTokenClassification, BertTokenizerFast

#use transformers of choice

MAX_LEN = 500
EPOCHS = 6
DEVICE = torch.device("cpu")
MODEL_PATH = '../input/bert-base-uncased'
STATE_DICT = torch.load('../working/model_e10.tar', map_location=DEVICE)
TOKENIZER = BertTokenizerFast('../input/bert-base-uncased/vocab.txt', lowercase=True)
MODEL = BertForTokenClassification.from_pretrained(MODEL_PATH, state_dict=STATE_DICT['model_state_dict'], num_labels=12)

In [ ]:
#700
tags_vals = ['Empty','UNKNOWN','Email Address', 'Links', 'Skills', 'Graduation Year', 'College Name', 'Degree', 'Companies worked at', 'Location', 'Name', 'Designation', 'projects', 'Years of Experience', 'Can Relocate to', 'Rewards and Achievements', 'Address', 'University', 'Relocate to', 'Certifications', 'state', 'links', 'College', 'training', 'des', 'abc']
tag2idx = {t: i for i, t in enumerate(tags_vals)}
idx2tag = {i:t for i, t in enumerate(tags_vals)}

In [ ]:
def process_resume2(text, tokenizer, max_len):
    tok = tokenizer.encode_plus(text, max_length=max_len, return_offsets_mapping=True)
    
    curr_sent = dict()
    
    padding_length = max_len - len(tok['input_ids'])
        
    curr_sent['input_ids'] = tok['input_ids'] + ([0] * padding_length)
    curr_sent['token_type_ids'] = tok['token_type_ids'] + ([0] * padding_length)
    curr_sent['attention_mask'] = tok['attention_mask'] + ([0] * padding_length)
    
    final_data = {
        'input_ids': torch.tensor(curr_sent['input_ids'], dtype=torch.long),
        'token_type_ids': torch.tensor(curr_sent['token_type_ids'], dtype=torch.long),
        'attention_mask': torch.tensor(curr_sent['attention_mask'], dtype=torch.long),
        'offset_mapping': tok['offset_mapping']
    }
    
    return final_data

In [ ]:
def predict(model, tokenizer, idx2tag, tag2idx, device, text):
    model.eval()
    data = process_resume2(text, tokenizer, MAX_LEN)
    input_ids, input_mask = data['input_ids'].unsqueeze(0), data['attention_mask'].unsqueeze(0)
    labels = torch.tensor([1] * input_ids.size(0), dtype=torch.long).unsqueeze(0)
    with torch.no_grad():
        outputs = model(
            input_ids,
            token_type_ids=None,
            attention_mask=input_mask,
            labels=labels,
        )
        tmp_eval_loss, logits = outputs[:2]
    
    logits = logits.cpu().detach().numpy()
    label_ids = np.argmax(logits, axis=2)
    
    entities = []
    for label_id, offset in zip(label_ids[0], data['offset_mapping']):
        curr_id = idx2tag[label_id]
        curr_start = offset[0]
        curr_end = offset[1]
        if curr_id != 'O':
            if len(entities) > 0 and entities[-1]['entity'] == curr_id and curr_start - entities[-1]['end'] in [0, 1]:
                entities[-1]['end'] = curr_end
            else:
                entities.append({'entity': curr_id, 'start': curr_start, 'end':curr_end})
    for ent in entities:
        ent['text'] = text[ent['start']:ent['end']]
    return entities

In [ ]:
entities1 = predict(model, TOKENIZER, idx2tag, tag2idx, DEVICE, text)

for i in entities1:
    print(i['entity'], '-', i['text'])